In [145]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

torch.manual_seed(1)

In [2]:
import json
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eliisabethein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# tokenizer = TweetTokenizer(preserve_case=False)
tok = MWETokenizer([('<', 'unk', '>')], separator = '')

train_data = []
vocabulary = []

with open("data/ptb.train.txt") as f:
    total_sentences = 0
    for i, line in enumerate(f):
        line = line.strip()
        s = word_tokenize(line)
        s = tok.tokenize(s)
        s = ['<sos>'] + s + ['<eos>']
        total_sentences += 1
        train_data.append(s)
        vocabulary += s

vocabulary = sorted(list(set(vocabulary)))
word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

print("total sentences:", total_sentences)

total sentences: 42068


In [6]:
print(vocabulary[0:100])

['#', '$', '&', "'", "'80s", "'d", "'ll", "'m", "'re", "'s", "'ve", '.', '10-year', '100-share', '12-month', '12-year', '13-week', '13th', '14-year-old', '190-point', '190.58-point', '1920s', '1930s', '1950s', '1960s', '1970s', '1980s', '1990s', '19th', '1\\/2-year', '2-for-1', '20-year', '20th', '24-hour', '26-week', '30-day', '30-share', '30-year', '300-a-share', '300-day', '40-year-old', '45-year-old', '500-stock', '52-week', '<eos>', '<sos>', '<unk>', 'N', '\\*', '\\*\\*', 'a', 'a.', 'a.c.', 'a.g.', 'a.m', 'a.m.', 'a.p', 'ab', 'aba', 'abandon', 'abandoned', 'abandoning', 'abbie', 'abc', 'ability', 'able', 'abm', 'aboard', 'abolish', 'abolished', 'aborted', 'abortion', 'abortion-rights', 'abortions', 'about', 'above', 'abrams', 'abramson', 'abroad', 'abrupt', 'abruptly', 'absence', 'absolutely', 'absorb', 'absorbed', 'absurd', 'abundant', 'abuse', 'abused', 'abuses', 'academic', 'academy', 'acadia', 'accelerate', 'accelerated', 'accelerating', 'acceleration', 'accept', 'acceptable',

In [7]:
from gensim.models import Word2Vec
model1 = Word2Vec(train_data, min_count=1, size=100, window=5)
model1.train(train_data, len(train_data), epochs=100)

(66894081, 97292600)

In [10]:
model1.wv.most_similar("stock")

[('shares', 0.6191723346710205),
 ('stocks', 0.5622472763061523),
 ('equity', 0.5172011852264404),
 ('share', 0.49241119623184204),
 ('mercantile', 0.4393764138221741),
 ('junk-bond', 0.42797455191612244),
 ('plunge', 0.4248238801956177),
 ('junk', 0.4115976393222809),
 ('listed', 0.4005429446697235),
 ('bancroft', 0.3902745246887207)]

In [11]:
model1.wv.most_similar(".")

[('f.', 0.44190406799316406),
 ('w.', 0.42585569620132446),
 ('s.', 0.4127725660800934),
 ('h.', 0.3979629874229431),
 ('mich.', 0.3972141742706299),
 ('n.j.', 0.3921316862106323),
 ('g.', 0.3899410665035248),
 ('ben', 0.3676188588142395),
 ('l.', 0.36253631114959717),
 ('neal', 0.3608734607696533)]

In [16]:
model1.wv.most_similar("credit")

[('debt', 0.4402936100959778),
 ('loans', 0.42133408784866333),
 ('financial', 0.4157423973083496),
 ('payment', 0.39422550797462463),
 ('coverage', 0.3736409842967987),
 ('deposit', 0.3729313910007477),
 ('write-downs', 0.35258758068084717),
 ('financing', 0.35087263584136963),
 ('receivables', 0.3490930497646332),
 ('asset-backed', 0.3486577272415161)]

In [17]:
model1.wv['credit']

array([ 8.7747359e-01,  2.9686937e+00,  2.6501117e+00,  3.0834761e+00,
        2.8429997e+00, -4.1582528e-01, -2.1958084e-01, -8.1865644e-01,
       -7.4580342e-01, -2.1322248e+00,  5.0131792e-01,  2.5824676e+00,
        1.0857420e+00, -3.9138109e-03, -1.7516010e+00,  1.2858166e+00,
       -7.7668393e-01, -1.9802961e+00,  1.4377120e+00,  5.3491254e+00,
        5.4552820e-02, -2.5157261e+00, -1.5339265e+00,  2.3392994e+00,
       -2.4587250e+00,  1.5426154e+00, -2.6625531e+00, -1.0871540e+00,
       -3.0938814e+00,  5.2244955e-01,  4.6642222e+00, -1.9233023e+00,
        2.1165483e+00,  4.5259926e-01,  2.7725346e+00, -2.7302454e+00,
        5.8359828e-02,  9.1397029e-01,  1.1368673e+00,  4.3198762e+00,
       -9.0990670e-02,  1.1243713e+00, -1.6051178e+00,  4.9419980e+00,
        6.8594003e-01,  1.2052360e+00,  2.9179436e-01, -2.6810260e+00,
       -6.8622929e-01,  2.9879980e+00, -1.6163086e+00, -9.4656420e-01,
       -6.3592833e-01,  6.2702984e-01,  2.0392424e-01,  2.7440622e+00,
      

In [137]:
vocabulary_size = len(vocabulary)
input_size = 100
output_size = 100
hidden_size = 10

In [81]:
input_word = 'money'
embedding_vec = model1.wv[input_word]

In [152]:
# make the word embeddings into a pythorch tensor
weights = torch.FloatTensor(model1.wv.vectors)

# NN MODEL
embedded = nn.Embedding.from_pretrained(weights) #input layer
lstm = nn.LSTM(input_size, hidden_size, num_layers=1) #lstm layer
out = nn.Linear(hidden_size, output_size)
softmax = nn.LogSoftmax(dim=1)

# loss_function = nn.NLLLoss()
# optimizer = optim.SGD( lr=0.1)

In [153]:
input_sentence = train_data[2]
input_idx = [model1.wv.vocab[x].index for x in input_sentence]

In [154]:
# intialise first hidden state
(hidden, cell) = (torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))

for idx in input_idx:
    print("input word:", model1.wv.index2word[idx])
    output = embedded(torch.tensor([idx], dtype=torch.long)).view(1, 1, 100)
    output, (hidden, cell) = lstm(output, (hidden, cell))
    output = softmax(out(output[0]))
    predicted_idx = np.argmax(output.detach().numpy())
    print("predicted_class: {0}".format(model1.wv.index2word[predicted_idx]))

input word: <sos>
predicted_class: after
input word: mr.
predicted_class: other
input word: <unk>
predicted_class: he
input word: is
predicted_class: into
input word: chairman
predicted_class: only
input word: of
predicted_class: he
input word: <unk>
predicted_class: he
input word: n.v.
predicted_class: into
input word: the
predicted_class: he
input word: dutch
predicted_class: he
input word: publishing
predicted_class: over
input word: group
predicted_class: <eos>
input word: <eos>
predicted_class: he
